In [14]:
import numpy as np
import h5py
import petl as etl
import allel; print('allel', allel.__version__)
%matplotlib inline
import matplotlib.pyplot as plt
import zarr
import seaborn as sns
sns.set_style('white')
sns.set_style('ticks')
import os
if 'DOCKER_IMAGE' in os.environ:
    print('docker image:', os.environ['DOCKER_IMAGE'])

allel 1.2.1


In [2]:
# set up sample metadata
tbl_samples_study = (
    etl
    .fromtsv('/media/sf_Data/Genomes/meta/meta_gambia.txt')
    .convert((3, 4, 5, 6, 7), float)
    .sort('ox_code')
)
tbl_samples_study

Sample_ID,ox_code,src_code,mean_coverage,pc_genome_covered,mapping_rate,mismatch_rate,duplicate_rate,ena_sample_acc,village,PCR,SINE200,Lat,Long,Village_IGS,Village_S200,Posizione,chrom X identification,karyotype 2R,2Rconf,2Rb,2Rc,2Rd,Inv2La,Taxon,Note,month
53624,AG0378-CW,001-1001,33.93,88.06,82.21,2.25,8.39,ERS680029,Tankular,M,na,13°25' N,16°02' W,Tankular_M,Tankular_na,Sud,A,+/+,-----,-,-,-,1,53624,14/06/2015,8
53625,AG0379-CW,001-1011,30.69,88.19,82.0,2.37,3.93,ERS680030,Tankular,M,na,13°25' N,16°02' W,Tankular_M,Tankular_na,Sud,A,b/b,-2---,2,-,-,2,53625,14/06/2015,8
53626,AG0380-CW,001-1032,35.08,88.02,82.33,2.35,9.18,ERS680031,Tankular,M,na,13°25' N,16°02' W,Tankular_M,Tankular_na,Sud,A,b/b,-2---,2,-,-,2,53626,14/06/2015,8
53627,AG0381-CW,001-1046,35.11,89.0,80.33,2.3,3.57,ERS680032,Tankular,M,na,13°25' N,16°02' W,Tankular_M,Tankular_na,Sud,A,b/b,-2---,2,-,-,2,53627,14/06/2015,8
53628,AG0382-CW,001-1048,36.75,89.56,84.65,2.32,2.18,ERS680033,Tankular,M,na,13°25' N,16°02' W,Tankular_M,Tankular_na,Sud,A,b/b,-2---,2,-,-,1,53628,14/06/2015,8


In [3]:
# this is the important bit - exclude samples with poor coverage
samples_study_pass = tbl_samples_study.gt('mean_coverage', 14).gt('pc_genome_covered', 88).values('ox_code').list()
len(samples_study_pass)

68

In [5]:
# set up sample metadata from Ag1000G phase 2 AR1
tbl_samples_ar1 = (
    etl
    .fromtsv('/media/sf_Data/Genomes/meta/samples.meta.txt')
    #.convert(0, int)
)
tbl_samples_ar1

ox_code,src_code,population,country,location,site,contributor,contact,year,m_s,sex,n_sequences,mean_coverage,ebi_sample_acc,latitude,longitude
AA0040-C,Twifo_Praso__E2,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95033368,30.99,ERS311878,5.60858,-1.54926
AA0041-C,Twifo_Praso__H3,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95843804,31.7,ERS311886,5.60858,-1.54926
AA0042-C,Takoradi_C7,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,107420666,35.65,ERS311894,4.91217,-1.77397
AA0043-C,Takoradi_H8,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,95993752,29.46,ERS311902,4.91217,-1.77397
AA0044-C,Takoradi_D10,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,103044262,33.67,ERS311910,4.91217,-1.77397


## Setup genotype data

In [6]:
# pick a chromosome
chrom = '3L'

In [8]:
# setup callset from the study
callset_study = h5py.File(
    '/media/sf_Data/Genomes/variation/gambia_hdf5/1130-AG-GM-CAPUTO.%s.h5' % chrom,
    mode='r'
)
callset_study

<HDF5 file "1130-AG-GM-CAPUTO.3L.h5" (mode r)>

In [11]:
# setup samples from the study
samples_study = [str(s, 'ascii') for s in callset_study[chrom]['samples'][:]]
samples_study[:5]

['AG0378-CW', 'AG0379-CW', 'AG0380-CW', 'AG0381-CW', 'AG0382-CW']

In [9]:
# setup genotypes from the study
genotypes_study = allel.GenotypeChunkedArray(callset_study[chrom]['calldata/genotype'])
genotypes_study

<GenotypeChunkedArray shape=(16437135, 109, 2) dtype=int8 chunks=(6553, 10, 2)
   nbytes=3.3G cbytes=239.3M cratio=14.3
   compression=gzip compression_opts=1
   values=h5py._hl.dataset.Dataset>

In [12]:
# subset to keep only the samples that pass our QC
sidx = [samples_study.index(s) for s in samples_study_pass]
genotypes_study_qc = genotypes_study.take(sidx, axis=1)
genotypes_study_qc

<GenotypeChunkedArray shape=(16437135, 68, 2) dtype=int8 chunks=(16052, 68, 2)
   nbytes=2.1G cbytes=253.1M cratio=8.4
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1, 'blocksize': 0}
   values=zarr.core.Array>

In [16]:
# setup callset from ag1000g phase 2
callset_ar3= zarr.open('/media/sf_Data/Genomes/variation/ag1000g.phase2.ar1.pass.biallelic/')
callset_ar3.tree()

/
 ├── 2L
 │   ├── samples (1142,) object
 │   └── variants
 │       ├── ABHet (8906423,) float32
 │       ├── ABHom (8906423,) float32
 │       ├── AC (8906423,) int32
 │       ├── AF (8906423,) float32
 │       ├── ALT (8906423,) |S1
 │       ├── AN (8906423,) int32
 │       ├── Accessible (8906423,) bool
 │       ├── BaseCounts (8906423, 4) int32
 │       ├── BaseQRankSum (8906423,) float32
 │       ├── Coverage (8906423,) int32
 │       ├── CoverageMQ0 (8906423,) int32
 │       ├── DP (8906423,) int32
 │       ├── DS (8906423,) bool
 │       ├── Dels (8906423,) float32
 │       ├── FILTER_BaseQRankSum (8906423,) bool
 │       ├── FILTER_FS (8906423,) bool
 │       ├── FILTER_HRun (8906423,) bool
 │       ├── FILTER_HighCoverage (8906423,) bool
 │       ├── FILTER_HighMQ0 (8906423,) bool
 │       ├── FILTER_LowCoverage (8906423,) bool
 │       ├── FILTER_LowMQ (8906423,) bool
 │       ├── FILTER_LowQual (8906423,) bool
 │       ├── FILTER_NoCoverage (8906423,) bool
 │       ├── FILTER_PASS (8906423,) bool
 │       ├── FILTER_QD (8906423,) bool
 │       ├── FILTER_ReadPosRankSum (8906423,) bool
 │       ├── FILTER_RefN (8906423,) bool
 │       ├── FILTER_RepeatDUST (8906423,) bool
 │       ├── FS (8906423,) float32
 │       ├── HRun (8906423,) int32
 │       ├── HW (8906423,) float32
 │       ├── HaplotypeScore (8906423,) float32
 │       ├── HighCoverage (8906423,) int32
 │       ├── HighMQ0 (8906423,) int32
 │       ├── InbreedingCoeff (8906423,) float32
 │       ├── LowCoverage (8906423,) int32
 │       ├── LowMQ (8906423,) int32
 │       ├── LowPairing (8906423,) int32
 │       ├── MLEAC (8906423,) int32
 │       ├── MLEAF (8906423,) float32
 │       ├── MQ (8906423,) float32
 │       ├── MQ0 (8906423,) int32
 │       ├── MQRankSum (8906423,) float32
 │       ├── NDA (8906423,) int32
 │       ├── NoCoverage (8906423,) int32
 │       ├── OND (8906423,) float32
 │       ├── POS (8906423,) int32
 │       ├── QD (8906423,) float32
 │       ├── QUAL (8906423,) float32
 │       ├── REF (8906423,) |S1
 │       ├── RPA (8906423,) int32
 │       ├── RU (8906423,) object
 │       ├── ReadPosRankSum (8906423,) float32
 │       ├── RefMasked (8906423,) bool
 │       ├── RefN (8906423,) bool
 │       ├── RepeatDUST (8906423,) bool
 │       ├── RepeatMasker (8906423,) bool
 │       ├── RepeatTRF (8906423,) bool
 │       ├── STR (8906423,) bool
 │       ├── VariantType (8906423,) object
 │       ├── altlen (8906423,) int32
 │       ├── is_snp (8906423,) bool
 │       └── numalt (8906423,) int32
 ├── 2R
 │   ├── samples (1142,) object
 │   └── variants
 │       ├── ABHet (12047846,) float32
 │       ├── ABHom (12047846,) float32
 │       ├── AC (12047846,) int32
 │       ├── AF (12047846,) float32
 │       ├── ALT (12047846,) |S1
 │       ├── AN (12047846,) int32
 │       ├── Accessible (12047846,) bool
 │       ├── BaseCounts (12047846, 4) int32
 │       ├── BaseQRankSum (12047846,) float32
 │       ├── Coverage (12047846,) int32
 │       ├── CoverageMQ0 (12047846,) int32
 │       ├── DP (12047846,) int32
 │       ├── DS (12047846,) bool
 │       ├── Dels (12047846,) float32
 │       ├── FILTER_BaseQRankSum (12047846,) bool
 │       ├── FILTER_FS (12047846,) bool
 │       ├── FILTER_HRun (12047846,) bool
 │       ├── FILTER_HighCoverage (12047846,) bool
 │       ├── FILTER_HighMQ0 (12047846,) bool
 │       ├── FILTER_LowCoverage (12047846,) bool
 │       ├── FILTER_LowMQ (12047846,) bool
 │       ├── FILTER_LowQual (12047846,) bool
 │       ├── FILTER_NoCoverage (12047846,) bool
 │       ├── FILTER_PASS (12047846,) bool
 │       ├── FILTER_QD (12047846,) bool
 │       ├── FILTER_ReadPosRankSum (12047846,) bool
 │       ├── FILTER_RefN (12047846,) bool
 │       ├── FILTER_RepeatDUST (12047846,) bool
 │       ├── FS (12047846,) float32
 │       ├── HRun (12047846,) int32
 │       ├── HW (12047846,) float32
 │       ├── HaplotypeScore (12047846,) float32
 │       ├── HighCoverage (12047846,) int32
 │       ├── HighMQ0 (12047846,) int32
 │

In [18]:
# setup genotypes from ag1000g phase 2
genotypes_ar3 = allel.GenotypeChunkedArray(callset_ar3[chrom]['calldata/GT'])
genotypes_ar3

<GenotypeChunkedArray shape=(7897666, 1142, 2) dtype=int8 chunks=(524288, 61, 2)
   nbytes=16.8G cbytes=256.9M cratio=67.0
   compression=blosc compression_opts={'cname': 'zstd', 'clevel': 1, 'shuffle': -1, 'blocksize': 0}
   values=zarr.core.Array>

## SNP ascertainment

In [19]:
# need the FILTER PASS values
variants_ar3 = allel.VariantChunkedTable(callset_ar3[chrom]['variants'],
                                         names=['POS', 'REF', 'ALT', 'FILTER_PASS'])
variants_ar3

<VariantChunkedTable shape=(7897666,) dtype=[('POS', '<i4'), ('REF', 'S1'), ('ALT', 'S1'), ('FILTER_PASS', '?')]
   nbytes=52.7M cbytes=15.1M cratio=3.5
   values=zarr.hierarchy.Group>

In [20]:
# count alleles in ag1000g
ac_ar3 = genotypes_ar3.count_alleles(max_allele=3)

KeyboardInterrupt: 

In [ ]:
# count alleles in study
ac_study = genotypes_study_qc.count_alleles(max_allele=3)

In [ ]:
# combine allele counts
ac = allel.AlleleCountsArray(ac_ar3 + ac_study)
ac

In [ ]:
# locate biallelic SNPs segregating above some reasonable frequency
loc_bi_seg = (ac.max_allele() <= 1) & (ac[:, :2].min(axis=1) > 7)
loc_bi_seg.size, np.count_nonzero(loc_bi_seg)